In [21]:
import html
import re
import httpx
from google import genai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
def get_summary(q):
    """Fetches a short summary from Wikipedia based on the search term."""
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })

    raw_snippet = response.json()["query"]["search"][0]["snippet"]

    # Remove HTML tags
    clean_snippet = re.sub(r'<[^>]+>', '', raw_snippet)

    # Unescape HTML entities (e.g., &quot; → ", &amp; → &)
    text_snippet = html.unescape(clean_snippet)

    return text_snippet

In [6]:
def quick_math(expr):
    """Evaluates a math expression given as a string."""
    return eval(expr)

In [8]:
def fetch_todo():
    """Fetches a placeholder to-do item from a fake API."""
    url = "https://jsonplaceholder.typicode.com/todos/1"
    res = httpx.get(url)

    if res.status_code == 200:
        return res.json()
    else:
        return {"error": "could not fetch"}

In [12]:
class Chat:
    """Wrapper class for Google's Gemini API"""
    
    def __init__(self, api_key, system=""):
        self.system = system
        self.messages = []
        self.genai_client = genai.Client(api_key=api_key)
        self.chat = self.genai_client.chats.create(model="gemini-2.0-flash")
        
        if self.system:
            self.messages.append({"role": "system", "content": system})
            self.chat.send_message(self.system)

    def __call__(self, message):
        if message:
            self.messages.append({"role": "user", "content": message})

        response = self.chat.send_message(message)
        result = response.text

        self.messages.append({"role": "assistant", "content": result})
        return result


In [22]:
test_chat = Chat(api_key=os.getenv("GEMINI_API_KEY"), system="You are a helpful assistant.")

In [23]:
test_chat("What is the capital of France?")

'The capital of France is **Paris**.\n'

In [ ]:
class ReActAgent:
    """
    A ReAct (Reasoning and Acting) agent that can use tools to answer questions.
    
    The agent follows the ReAct pattern:
    1. Reason about the task
    2. Act by calling appropriate tools
    3. Observe the results
    4. Repeat until the task is complete
    """
    
    def __init__(self, api_key):
        self.memory = []
        self.system_prompt = (
            "You are a helpful assistant. You can use the following tools:\n"
            "- get_summary(query: str): to search Wikipedia\n"
            "- quick_math(expr: str): to evaluate a math expression\n"
            "- fetch_todo(): to get a sample todo\n"
            "When you need to use a tool, respond with: Action: <tool_name>[<input>]\n"
            "Otherwise, respond normally as the assistant.\n"
        )
        
        self.chat = Chat(api_key, system=self.system_prompt)

    def __call__(self, message):
        """Process a user message and return a response, using tools if necessary."""
        full_input = "\n".join(self.memory + [f"User: {message}"])
        
        response = self.chat(full_input)
        
        self.memory.append(f"User: {message}")
        self.memory.append(f"Assistant: {response}")

        # Check if the response contains a tool invocation
        if response.startswith("Action:"):
            tool_call = re.match(r'Action:\s*(\w+)\[(.*?)\]', response)
            if not tool_call:
                return "Invalid tool format."

            tool_name, tool_arg = tool_call.groups()
            result = self.invoke_tool(tool_name, tool_arg.strip())
            
            self.memory.append(f"Observation: {result}")
            
            # Recursive call to process the tool result
            return self(f"Observation: {result}")
        else:
            return response

    def invoke_tool(self, name, arg):
        """Execute a tool with the given argument."""
        try:
            if name == "get_summary":
                return get_summary(arg)
            elif name == "quick_math":
                return quick_math(arg)
            elif name == "fetch_todo":
                return fetch_todo([])
            else:
                return f"Unknown tool: {name}"
        except Exception as e:
            return f"Error executing {name}: {str(e)}"


In [30]:
def main():
    API_KEY = os.getenv("GEMINI_API_KEY")  
    agent = ReActAgent(API_KEY)
    
    print("Welcome to the ReAct Agent! Type 'exit' to quit.")
    print(agent("Hello!"))
    
    print(agent("What is the capital of France?"))
    print(agent("What is 2 + 2?"))
    



if __name__ == "__main__":
    main()

Welcome to the ReAct Agent! Type 'exit' to quit.
Hello there! How can I help you today?

The capital of France is Paris.

The answer to 2 + 2 is 4.

